In [1]:
!pip install openai

# 고1 데이터

### data preprocessing

In [2]:
import pandas as pd
import numpy as np
import os
import json

In [3]:
#고1
folder_path_1 = "/Users/seunghyun/Library/CloudStorage/OneDrive-고려대학교/024.에세이 글 평가 데이터/주장/고1"

data_1 = []

for filename in os.listdir(folder_path_1):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path_1, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
            data_1.append(json_data)

print(f'{len(data_1)}개의 파일이 성공적으로 읽혔습니다.')

359개의 파일이 성공적으로 읽혔습니다.


In [4]:
data_list_1 = []

for filename in os.listdir(folder_path_1):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path_1, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)

            # 필요한 정보 추출
            paragraph_txt = json_data['paragraph'][0]['paragraph_txt']
            essay_prompt = json_data['info']['essay_prompt']
            essay_len = json_data['info']['essay_len']
            essay_main_subject = json_data['info']['essay_main_subject']

            # 데이터를 리스트에 추가
            data_list_1.append({
                'paragraph_txt': paragraph_txt,
                'essay_prompt': essay_prompt,
                'essay_len': essay_len,
                'essay_main_subject': essay_main_subject
            })

# 리스트를 데이터프레임으로 변환
df_1 = pd.DataFrame(data_list_1)

In [5]:
df_1['essay_len'].describe()

count     359.000000
mean      869.325905
std       132.962830
min       302.000000
25%       810.000000
50%       845.000000
75%       910.500000
max      1601.000000
Name: essay_len, dtype: float64

- 고1 분량 1000자 정도

In [6]:
import re

# @#문장구분# 이 포함된 부분 삭제
df_1['paragraph_txt'] = df_1['paragraph_txt'].replace(r'#@문장구분#', '', regex=True)

df_1.head()

,paragraph_txt,essay_prompt,essay_len,essay_main_subject
0,저희 학교는 남녀공학입니다. 저번 체육시간에 원반던지기 평가가 있었습니다. 각 모둠...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,1135,평가에 대한 본인의 의견
1,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,850,평가에 대한 본인의 의견
2,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,817,평가에 대한 본인의 의견
3,평가란 평가대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가지표...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,816,평가에 대한 본인의 의견
4,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘...",평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,818,평가에 대한 본인의 의견


In [8]:
df_1['essay_prompt'][0]

' 평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상에는 무수히 많은 평가 방법들이 존재하고 그 주제도 다양합니다.\n\n 우리나라의 경우는 어떨까요? 중간, 기말시험이 있고 미술과 같은 기본기와 창의성을 평가하는 과목도 있습니다. 체육과 같은 신체적인 능력을 평가하는 과목도 있지요. 모두 적절한 기준이 있으며 매우 객관적인 기준이라고 말합니다.\n\n 하지만 여러분이 생각하기에 모든 학생을 동일한 평가지표에 맞추어 평가하는 것이 올바른 방식이라고 생각하나요?\n\n 사람마다 특성이 존재하듯 각각의 목표 또한 다르고 성과도 다를 수 있습니다. 특정 기준에 의존해 본인의 성과도를 평가하는 것이 바르다고 생각되나요?\n\n 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하신다면 그 이유를 작성해주세요. 아니라면 어떤 평가 기준이 추가되어야 하고 또한 어떤 생각으로 평가가 이루어졌으면 좋겠는지 서술해 주세요.'

In [7]:
df_1.to_csv('고1essay.csv')

## Gpt4o-mini setting

In [ ]:
from openai import OpenAI
import os

## Set the API key and model name
MODEL="gpt-4o-mini"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "YOUR_KEY"))

In [8]:
completion = client.chat.completions.create(
  model=MODEL,
  messages=[
    {"role": "system", "content": "You are a helpful assistant. Help me with my math homework!"}, # <-- This is the system message that provides context to the model
    {"role": "user", "content": "Hello! Could you solve 2+2?"}  # <-- This is the user message for which the model will generate a response
  ]
)

print("Assistant: " + completion.choices[0].message.content)

Assistant: Of course! \( 2 + 2 = 4 \).


## 주장_고등1학년

### 고1_paraphrase
-  **high1 데이터**

In [9]:
essay1 = pd.read_csv("/Users/seunghyun/Downloads/고1essay.csv")

In [10]:
essay1

,Unnamed: 0,paragraph_txt,essay_prompt,essay_len,essay_main_subject
0,0,저희 학교는 남녀공학입니다. 저번 체육시간에 원반던지기 평가가 있었습니다. 각 모둠...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,1135,평가에 대한 본인의 의견
1,1,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,850,평가에 대한 본인의 의견
2,2,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,817,평가에 대한 본인의 의견
3,3,평가란 평가대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가지표...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,816,평가에 대한 본인의 의견
4,4,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘...",평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,818,평가에 대한 본인의 의견
...,...,...,...,...,...
354,354,오늘은 많은 평가 중에서 교육 평가에 대해서 얘기해보려고 한다. 교육평가의 기능 중...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,801,평가에 대한 본인의 의견
355,355,장점과 가치를 결정하는 과정을 평가라고 하였을 때 이 평가는 우리에게 굉장히 중요한...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,866,평가에 대한 본인의 의견
356,356,안녕하세요. 저는 우리나라의 평가 방식이 다소 잘못된 방향으로 흘러가고 있다고 생각...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,822,평가에 대한 본인의 의견
357,357,저는 글을 쓰는 주제로 우리가 학교나 사회 생활을 하며 흔하게 겪었을 법한 '평가'...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,813,평가에 대한 본인의 의견


In [11]:
high1 = essay1[:100]
len(high1)

100

In [12]:
high1.head()

,Unnamed: 0,paragraph_txt,essay_prompt,essay_len,essay_main_subject
0,0,저희 학교는 남녀공학입니다. 저번 체육시간에 원반던지기 평가가 있었습니다. 각 모둠...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,1135,평가에 대한 본인의 의견
1,1,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,850,평가에 대한 본인의 의견
2,2,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,817,평가에 대한 본인의 의견
3,3,평가란 평가대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가지표...,평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,816,평가에 대한 본인의 의견
4,4,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘...",평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.\n\n 세상...,818,평가에 대한 본인의 의견


In [15]:
paraphrase_prompt = '''
아래의 글을 정제된 언어로 paraphrase 해줘.
--------------------
{text}
'''

In [47]:
original_texts = []
paraphrased_texts = []

# 각 텍스트에 대해 paraphrase 실행
for text in essay1['paragraph_txt']:
    # 텍스트를 paraphrase_prompt에 삽입
    user_prompt = paraphrase_prompt.format(text=text)

    # API 요청 생성
    completion = client.chat.completions.create(
      model=MODEL,
      messages=[
        {"role": "system", "content": paraphrase_prompt},
        {"role": "user", "content": user_prompt}
      ]
    )

    # 결과 저장
    original_texts.append(text)
    paraphrased_texts.append(completion.choices[0].message.content)

# DataFrame 생성
df = pd.DataFrame({
    'Original Text': original_texts,
    'Paraphrased Text': paraphrased_texts
})
df.to_csv("paraphrase_high1")

In [49]:
len(df)

359

In [50]:
print(df['Original Text'][3])

평가란 평가대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가지표에 맞줘 평가한는 건 옳치 않다 생각한다. 교육평가의 정의를 목표달성도 확인, 의사결정을 위한 정보제공, 그리고 가치의 판단으로 구분할 수 있다. 이 관점은 가치판단적 의미를 축소하거나 배제시키고 교육평가를 교육에 관한 의사결정을 내릴 때 필요한 정보를 수집해서 의사결정에 활용하는 과정으로 부각시키려고 한다. 입장에서는 교육평가의 과정을 우선평가기준을 설정하고 관련 정보를 수집한 뒤에 평가기준을 적용하여 대상의 가치나 유용성, 효과, 중요성 등을 판단하는 것으로 생각한다. 더욱이 보상,채용,교육,훈련등 다른 프로세스와의 밀접한 상호 관련성을 고려하여 평가 제도를 설계해야 한다는 점도 평가자에 있어서는 큰 부담이다. 그럼에도 불구하고 올바른 성과주의 문화를 정착시키기 위해서는 공정하고 객관적인 평가 시스템의 구축이 반드시 필요하다. 평가에 대해 갖고 있는 오해를 불식시키고 바른 해결점을 찾아가는 과정이 뒷받침 되어야 좋은 평가 시스템이 얻어질수 있는 것 같다. 따라서 이 입장에서는 의사결정에 필요하다면 교육의 성과뿐만 아니라 정책, 제도, 시설, 재정 등과 같이 교육에 투입되고 운영되는 투입과 과정에 관한 정보도 수집하여 활용할 것을 강조한다. 평가라는 시스템이 사람을 얼마나 비참하게 만드는 지를 알고 지나간 평가는 다시 되돌릴 수 없기에 내가 생각하는 평가라는 지표는 인생에 매우 필요한 것이기도 하며 우리의 삶이라는 초원을 한 순간 불바다로 만들 수 있는 작은 불씨라고 할 수 있다. 하지만 이에 무반응하지 않고 개선하려는 노력 또한 필요할 것으로 보인다.


In [51]:
print(df['Paraphrased Text'][3])

평가는 평가대상의 강점과 가치를 판별하는 과정입니다. 학생들에게 동일한 기준으로 평가하는 것은 바람직하지 않다고 생각합니다. 교육평가는 목표 달성 여부 확인, 의사결정을 위한 정보 제공, 가치 판단으로 정의될 수 있습니다. 이러한 관점은 교육평가의 가치 판단적 의미를 축소하거나 제외하고, 교육에 관한 의사결정을 내리기 위해 필요한 정보를 수집하고 이를 의사결정에 활용하는 과정으로 강조됩니다. 따라서 교육평가 과정은 먼저 평가 기준을 설정하고, 관련 정보를 수집한 뒤 평가 기준을 적용하여 대상의 가치나 유용성, 효과성, 중요성 등을 판단하는 것으로 이해됩니다. 또한 보상, 채용, 교육, 훈련 등 다른 프로세스와의 밀접한 연계성을 고려하여 평가 체계를 설계해야 한다는 점은 평가자에게 큰 부담으로 작용합니다. 그럼에도 불구하고 올바른 성과 중심 문화를 정착시키기 위해서는 공정하고 객관적인 평가 시스템이 꼭 필요합니다. 평가에 대한 오해를 해소하고 적절한 해결 방안을 찾는 과정이 좋은 평가 시스템의 구축에 기여할 것입니다. 따라서 이 관점에서는 교육 성과뿐만 아니라 정책, 제도, 시설, 재정 등 교육 운영과 관련된 정보도 수집하여 활용할 필요성이 강조됩니다. 평가라는 시스템이 개인에게 미치는 부정적인 영향을 인식하고, 과거의 평가는 되돌릴 수 없으므로 제가 생각하는 평가는 인생에서 매우 중요하며, 우리의 삶을 한순간에 변화시킬 수 있는 작은 요소라고 할 수 있습니다. 그러나 이에 대해 무관심하지 않고 개선을 위한 노력이 필요합니다.


paraphrase_high1 = original text + paraphrased text

### 고1_맞춤법 검사
-  **high1 데이터**

In [52]:
correction_prompt = '''
아래의 글에서 띄어쓰기 및 맞춤법 오류가 있는지 확인해보고, 있는 경우 수정해줘. 그리고 수정된 글을 반환해줘.
하나라도 수정한 글에는 글 앞에 '아래 글에서 맞춤법 오류를 수정한 내용입니다.\n\n-----'를 적어줘.
--------------------
{text}
'''

In [53]:
original_texts = []
correction_texts = []

# 각 텍스트에 대해 paraphrase 실행
for text in essay1['paragraph_txt']:
    # 텍스트를 paraphrase_prompt에 삽입
    user_prompt = correction_prompt.format(text=text)

    # API 요청 생성
    completion = client.chat.completions.create(
      model=MODEL,
      messages=[
        {"role": "system", "content": correction_prompt},
        {"role": "user", "content": user_prompt}
      ],

    temperature = 0.7
    )

    # 결과 저장
    original_texts.append(text)
    correction_texts.append(completion.choices[0].message.content)

# DataFrame 생성
high1_df2 = pd.DataFrame({
    'Original Text': original_texts,
    'Paraphrased Text': correction_texts
})

high1_df2['Paraphrased Text'] = high1_df2['Paraphrased Text'].str.replace(r'아래 글에서 맞춤법 오류를 수정한 내용입니다.\n\n-----', '', regex=True)

high1_df2.to_csv("correction_high1")

In [54]:
# 컬럼 이름 변경
high1_df2.rename(columns={'Paraphrased Text': 'Correction Text'}, inplace=True)

# 결과 저장
high1_df2.to_csv("correction_high1.csv", index=False)  # 확장자 추가 및 index=False 설정

In [59]:
high1_df2.iloc[135]

Original Text      현재 우리학교생할은 학력중심사회로 우리는 고등학교과정을 대학을 가기위한 전과정으로 ...
Correction Text    \n\n현재 우리 학교 생활은 학력 중심 사회로 우리는 고등학교 과정을 대학을 가기...
Name: 135, dtype: object

In [60]:
# Correction Text 컬럼에서 \n 제거
high1_df2['Correction Text'] = high1_df2['Correction Text'].str.replace(r'\n\n', '', regex=True)

# 결과 저장
high1_df2.to_csv("correction_high1.csv", index=False)

In [62]:
high1_df2

,Original Text,Correction Text
0,저희 학교는 남녀공학입니다. 저번 체육시간에 원반던지기 평가가 있었습니다. 각 모둠...,저희 학교는 남녀공학입니다. 저번 체육 시간에 원반 던지기 평가가 있었습니다. 각 ...
1,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...
2,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...
3,평가란 평가대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가지표...,평가란 평가 대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가 ...
4,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘...","위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘..."
...,...,...
354,오늘은 많은 평가 중에서 교육 평가에 대해서 얘기해보려고 한다. 교육평가의 기능 중...,오늘은 많은 평가 중에서 교육 평가에 대해서 얘기해보려고 한다. 교육 평가의 기능 ...
355,장점과 가치를 결정하는 과정을 평가라고 하였을 때 이 평가는 우리에게 굉장히 중요한...,장점과 가치를 결정하는 과정을 평가라고 하였을 때 이 평가는 우리에게 굉장히 중요한...
356,안녕하세요. 저는 우리나라의 평가 방식이 다소 잘못된 방향으로 흘러가고 있다고 생각...,안녕하세요. 저는 우리나라의 평가 방식이 다소 잘못된 방향으로 흘러가고 있다고 생각...
357,저는 글을 쓰는 주제로 우리가 학교나 사회 생활을 하며 흔하게 겪었을 법한 '평가'...,저는 글을 쓰는 주제로 우리가 학교나 사회 생활을 하며 흔하게 겪었을 법한 '평가'...


### 고1_생성 (에세이 주제만 가지고)
-  **high1 데이터**

In [63]:
essay_prompt_high1 = '''
평가는 일반적으로 평가대상의 장점과 가치를 결정하는 과정을 말합니다.
세상에는 무수히 많은 평가 방법들이 존재하고 그 주제도 다양합니다.
우리나라의 경우는 어떨까요? 중간, 기말시험이 있고 미술과 같은 기본기와 창의성을 평가하는 과목도 있습니다. 체육과 같은 신체적인 능력을 평가하는 과목도 있지요. 모두 적절한 기준이 있으며 매우 객관적인 기준이라고 말합니다.
하지만 여러분이 생각하기에 모든 학생을 동일한 평가지표에 맞추어 평가하는 것이 올바른 방식이라고 생각하나요?
사람마다 특성이 존재하듯 각각의 목표 또한 다르고 성과도 다를 수 있습니다. 특정 기준에 의존해 본인의 성과도를 평가하는 것이 바르다고 생각되나요?
현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하신다면 그 이유를 작성해주세요. 아니라면 어떤 평가 기준이 추가되어야 하고 또한 어떤 생각으로 평가가 이루어졌으면 좋겠는지 서술해 주세요.
'''

In [64]:
agent_prompt_high1 = '''
너는 고등학교 3학년 학생이고, 주장과 그에 대한 근거를 높은 수준에서 제시할 수 있어.
다음 주제로 1000자 글짓기를 하고 있어. 주제에 대해 너의 근거를 들어 주장하는 글쓰기를 작성해줘.
제목은 작성하지 않고, 본문만 작성해줘.
'''

In [67]:
import pandas as pd

paraphrase_high1 = pd.read_csv('/Users/seunghyun/Downloads/paraphrase_high1.csv')  
correction_high1 = pd.read_csv('/Users/seunghyun/Downloads/correction_high1.csv')  

merged_df = pd.merge(paraphrase_high1, correction_high1, on='Original Text', how='inner')

In [68]:
merged_df.to_csv("merged_high1.csv", index=False)

In [70]:
len(merged_df)

361

In [71]:
merged_df.head()

,Unnamed: 0,Original Text,Paraphrased Text,Correction Text
0,0,저희 학교는 남녀공학입니다. 저번 체육시간에 원반던지기 평가가 있었습니다. 각 모둠...,저희 학교는 남녀공학입니다. 최근 체육 시간에 원반던지기 평가가 있었습니다. 각 모...,저희 학교는 남녀공학입니다. 저번 체육 시간에 원반 던지기 평가가 있었습니다. 각 ...
1,1,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...,현재의 평가 방식에 새로운 평가 기준을 추가할 필요가 있다고 생각합니다. 평가는 대...,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...
2,2,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...,현재의 평가 방식은 최고의 객관적인 방법이라고 생각하지 않습니다. 그 이유를 설명하...,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...
3,3,평가란 평가대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가지표...,평가는 평가대상의 강점과 가치를 판별하는 과정입니다. 학생들에게 동일한 기준으로 평...,평가란 평가 대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가 ...
4,4,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘...",위에서 언급한 바와 같이 평가는 대상을 분석하여 그 가치를 파악하는 과정입니다. 그...,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘..."


In [72]:
merged_df.tail()

,Unnamed: 0,Original Text,Paraphrased Text,Correction Text
356,354,오늘은 많은 평가 중에서 교육 평가에 대해서 얘기해보려고 한다. 교육평가의 기능 중...,오늘은 다양한 평가 분야 중 교육 평가에 대해 논의하고자 합니다. 교육 평가의 기능...,오늘은 많은 평가 중에서 교육 평가에 대해서 얘기해보려고 한다. 교육 평가의 기능 ...
357,355,장점과 가치를 결정하는 과정을 평가라고 하였을 때 이 평가는 우리에게 굉장히 중요한...,"장점과 가치를 결정하는 과정을 평가라고 정의할 때, 이 평가는 우리의 삶에서 매우 ...",장점과 가치를 결정하는 과정을 평가라고 하였을 때 이 평가는 우리에게 굉장히 중요한...
358,356,안녕하세요. 저는 우리나라의 평가 방식이 다소 잘못된 방향으로 흘러가고 있다고 생각...,안녕하세요. 저는 우리나라의 평가 방식이 다소 잘못된 방향으로 진행되고 있다고 생각...,안녕하세요. 저는 우리나라의 평가 방식이 다소 잘못된 방향으로 흘러가고 있다고 생각...
359,357,저는 글을 쓰는 주제로 우리가 학교나 사회 생활을 하며 흔하게 겪었을 법한 '평가'...,저는 글의 주제로 우리가 학교와 사회에서 흔히 경험하는 '평가'에 대한 제 생각을 ...,저는 글을 쓰는 주제로 우리가 학교나 사회 생활을 하며 흔하게 겪었을 법한 '평가'...
360,358,현대사회는 학력중심사회로 우리는 고등학교과정을 대학을 가기위한 전과정으로 생각하는 ...,"현대 사회는 학력을 중시하는 경향이 강해, 많은 사람들이 고등학교 과정을 대학 진학...",현대사회는 학력 중심 사회로 우리는 고등학교 과정을 대학을 가기 위한 전 과정으로 ...


In [77]:
merged_df = merged_df.drop('Unnamed: 0', axis=1)

In [78]:
merged_df

,Original Text,Paraphrased Text,Correction Text
0,저희 학교는 남녀공학입니다. 저번 체육시간에 원반던지기 평가가 있었습니다. 각 모둠...,저희 학교는 남녀공학입니다. 최근 체육 시간에 원반던지기 평가가 있었습니다. 각 모...,저희 학교는 남녀공학입니다. 저번 체육 시간에 원반 던지기 평가가 있었습니다. 각 ...
1,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...,현재의 평가 방식에 새로운 평가 기준을 추가할 필요가 있다고 생각합니다. 평가는 대...,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...
2,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...,현재의 평가 방식은 최고의 객관적인 방법이라고 생각하지 않습니다. 그 이유를 설명하...,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...
3,평가란 평가대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가지표...,평가는 평가대상의 강점과 가치를 판별하는 과정입니다. 학생들에게 동일한 기준으로 평...,평가란 평가 대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가 ...
4,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘...",위에서 언급한 바와 같이 평가는 대상을 분석하여 그 가치를 파악하는 과정입니다. 그...,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘..."
...,...,...,...
356,오늘은 많은 평가 중에서 교육 평가에 대해서 얘기해보려고 한다. 교육평가의 기능 중...,오늘은 다양한 평가 분야 중 교육 평가에 대해 논의하고자 합니다. 교육 평가의 기능...,오늘은 많은 평가 중에서 교육 평가에 대해서 얘기해보려고 한다. 교육 평가의 기능 ...
357,장점과 가치를 결정하는 과정을 평가라고 하였을 때 이 평가는 우리에게 굉장히 중요한...,"장점과 가치를 결정하는 과정을 평가라고 정의할 때, 이 평가는 우리의 삶에서 매우 ...",장점과 가치를 결정하는 과정을 평가라고 하였을 때 이 평가는 우리에게 굉장히 중요한...
358,안녕하세요. 저는 우리나라의 평가 방식이 다소 잘못된 방향으로 흘러가고 있다고 생각...,안녕하세요. 저는 우리나라의 평가 방식이 다소 잘못된 방향으로 진행되고 있다고 생각...,안녕하세요. 저는 우리나라의 평가 방식이 다소 잘못된 방향으로 흘러가고 있다고 생각...
359,저는 글을 쓰는 주제로 우리가 학교나 사회 생활을 하며 흔하게 겪었을 법한 '평가'...,저는 글의 주제로 우리가 학교와 사회에서 흔히 경험하는 '평가'에 대한 제 생각을 ...,저는 글을 쓰는 주제로 우리가 학교나 사회 생활을 하며 흔하게 겪었을 법한 '평가'...


In [79]:
merged2 = merged_df[:100]

In [80]:
len(merged2)

100

In [81]:
merged2

,Original Text,Paraphrased Text,Correction Text
0,저희 학교는 남녀공학입니다. 저번 체육시간에 원반던지기 평가가 있었습니다. 각 모둠...,저희 학교는 남녀공학입니다. 최근 체육 시간에 원반던지기 평가가 있었습니다. 각 모...,저희 학교는 남녀공학입니다. 저번 체육 시간에 원반 던지기 평가가 있었습니다. 각 ...
1,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...,현재의 평가 방식에 새로운 평가 기준을 추가할 필요가 있다고 생각합니다. 평가는 대...,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...
2,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...,현재의 평가 방식은 최고의 객관적인 방법이라고 생각하지 않습니다. 그 이유를 설명하...,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...
3,평가란 평가대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가지표...,평가는 평가대상의 강점과 가치를 판별하는 과정입니다. 학생들에게 동일한 기준으로 평...,평가란 평가 대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가 ...
4,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘...",위에서 언급한 바와 같이 평가는 대상을 분석하여 그 가치를 파악하는 과정입니다. 그...,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘..."
...,...,...,...
95,나는 수학능력시험에 대해 정의를 실현한다고 생각하지 않는다. 왜냐하면 이런 평가는 ...,나는 수학능력시험이 정의를 실현한다고 보지 않는다. 이는 조선시대의 과거제와 유사하...,\n나는 수학능력시험에 대해 정의를 실현한다고 생각하지 않는다. 왜냐하면 이런 평가...
96,나는 우리나라 수학능력평가시험이 현 입시의 평가인 일명 수능으로만 평가를 하여 대학...,저는 우리나라의 수학능력평가시험이 대학교 입학 여부를 결정하는 유일한 기준으로 사용...,나는 우리나라 수학능력평가시험이 현 입시의 평가인 일명 수능으로만 평가를 하여 대학...
97,평가는 학생 뿐만 아니라 태어나면서 부터 성인이 돼서 사회에 나가서도 계속되는게 평...,평가는 학생뿐만 아니라 인간이 태어나 성인이 되어 사회에 나가는 동안 지속되는 중요...,평가는 학생 뿐만 아니라 태어나면서부터 성인이 돼서 사회에 나가서도 계속되는 게 평...
98,저는 현재 진행되고 있는 평가 방식이 가장 객관적인 방식이라고 생각합니다. 왜냐하면...,현재 시행되고 있는 평가 방식이 가장 객관적이라고 생각합니다. 평가란 본질적으로 평...,저는 현재 진행되고 있는 평가 방식이 가장 객관적인 방식이라고 생각합니다. 왜냐하면...


In [82]:
#generation

generated_texts = []
for text in essay1['paragraph_txt'][:100]:  
    completion = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": agent_prompt_high1},
            {"role": "user", "content": essay_prompt_high1}
        ],
        temperature=1.0
    )
    generated_texts.append(completion.choices[0].message.content)

# 병합된 데이터프레임에 Generated Texts 컬럼 추가
merged2['Generated Texts'] = generated_texts

# 저장 또는 데이터 확인
merged2.to_csv('merged_high1.csv', index=False) 

/var/folders/z9/csz01p_s7kj4t80shshdpg1w0000gn/T/ipykernel_8821/2250006245.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged2['Generated Texts'] = generated_texts


In [84]:
# \n 모두 제거

# 모든 컬럼의 텍스트에서 '\n' 제거
merged2 = merged2.applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)

# 변경된 DataFrame 저장 (선택 사항)
merged2.to_csv('merged_high1.csv', index=False)

/var/folders/z9/csz01p_s7kj4t80shshdpg1w0000gn/T/ipykernel_8821/3615906425.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged2 = merged2.applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)


In [88]:
merged2

,Original Text,Paraphrased Text,Correction Text,Generated Texts
0,저희 학교는 남녀공학입니다. 저번 체육시간에 원반던지기 평가가 있었습니다. 각 모둠...,저희 학교는 남녀공학입니다. 최근 체육 시간에 원반던지기 평가가 있었습니다. 각 모...,저희 학교는 남녀공학입니다. 저번 체육 시간에 원반 던지기 평가가 있었습니다. 각 ...,"현재의 평가 방식은 학습의 성과와 학생의 특성을 충분히 반영하지 못하고 있으며, 이..."
1,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...,현재의 평가 방식에 새로운 평가 기준을 추가할 필요가 있다고 생각합니다. 평가는 대...,저는 현재 우리의 평가 방식에서 새로운 평가 기준이 추가되어야 한다고 생각합니다. ...,현재의 평가 방식은 학생 개개인의 다양성을 충분히 반영하고 있지 못하다고 생각한다....
2,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...,현재의 평가 방식은 최고의 객관적인 방법이라고 생각하지 않습니다. 그 이유를 설명하...,저는 현재의 평가 방식은 최고의 객관적인 평가 방식이라고 생각하지 않아요. 그 이유...,"현재의 교육 평가 방식에는 분명 장점이 존재하지만, 이를 전면적으로 옹호하기에는 여..."
3,평가란 평가대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가지표...,평가는 평가대상의 강점과 가치를 판별하는 과정입니다. 학생들에게 동일한 기준으로 평...,평가란 평가 대상의 장점과 가치를 결정하는 과정을 말한다. 학생들에게 동일한 평가 ...,현재의 평가 방식이 모든 학생에게 공정하고 효과적이라고 보기는 어렵다. 우리나라의 ...
4,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘...",위에서 언급한 바와 같이 평가는 대상을 분석하여 그 가치를 파악하는 과정입니다. 그...,"위에서 말한 것처럼 평가는 대상의 장점이나 가치를 결정하는 과정이기도 하지만, 요즘...",현재의 교육 평가 방식이 과연 모든 학생의 개성과 잠재력을 충분히 반영하고 있는지에...
...,...,...,...,...
95,나는 수학능력시험에 대해 정의를 실현한다고 생각하지 않는다. 왜냐하면 이런 평가는 ...,나는 수학능력시험이 정의를 실현한다고 보지 않는다. 이는 조선시대의 과거제와 유사하...,나는 수학능력시험에 대해 정의를 실현한다고 생각하지 않는다. 왜냐하면 이런 평가는 ...,현행 교육 시스템에서 시행되는 평가 방법에 대해 진지한 성찰이 필요하다. 전통적인 ...
96,나는 우리나라 수학능력평가시험이 현 입시의 평가인 일명 수능으로만 평가를 하여 대학...,저는 우리나라의 수학능력평가시험이 대학교 입학 여부를 결정하는 유일한 기준으로 사용...,나는 우리나라 수학능력평가시험이 현 입시의 평가인 일명 수능으로만 평가를 하여 대학...,현재의 평가 방식은 다양한 특성과 목표를 가진 학생들을 모두 동일한 기준으로 평가하...
97,평가는 학생 뿐만 아니라 태어나면서 부터 성인이 돼서 사회에 나가서도 계속되는게 평...,평가는 학생뿐만 아니라 인간이 태어나 성인이 되어 사회에 나가는 동안 지속되는 중요...,평가는 학생 뿐만 아니라 태어나면서부터 성인이 돼서 사회에 나가서도 계속되는 게 평...,"현재의 평가 방식에는 분명한 장점이 있지만, 모든 학생을 동일한 기준으로 평가하는 ..."
98,저는 현재 진행되고 있는 평가 방식이 가장 객관적인 방식이라고 생각합니다. 왜냐하면...,현재 시행되고 있는 평가 방식이 가장 객관적이라고 생각합니다. 평가란 본질적으로 평...,저는 현재 진행되고 있는 평가 방식이 가장 객관적인 방식이라고 생각합니다. 왜냐하면...,현재의 평가 방식은 학생들의 다양한 재능과 특성을 반영하지 못하는 한계가 있다는 점...


### to_json!

In [94]:
# paraphrased 전용

csv_path = '/Users/seunghyun/Downloads/merged_high1.csv'
merged_df = pd.read_csv(csv_path)

# JSON 파일 형식 생성
json_output = []

# CSV의 각 행을 반복하며 JSON 포맷으로 변환
json_output = {
    "original": merged_df['Correction Text'].tolist(),
    "sampled": merged_df['Paraphrased Text'].tolist()
}


# JSON 파일 저장
output_path = 'data_paraphrased.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(json_output, f, ensure_ascii=False, indent=4)

print(f"JSON 파일이 생성되었습니다: {output_path}")

JSON 파일이 생성되었습니다: data_paraphrased.json


In [93]:
# generation 전용

csv_path = '/Users/seunghyun/Downloads/merged_high1.csv'
merged_df = pd.read_csv(csv_path)

# JSON 파일 형식 생성
json_output = []

# CSV의 각 행을 반복하며 JSON 포맷으로 변환
json_output = {
    "original": merged_df['Correction Text'].tolist(),
    "sampled": merged_df['Generated Texts'].tolist()
}


# JSON 파일 저장
output_path = 'data_generated.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(json_output, f, ensure_ascii=False, indent=4)

print(f"JSON 파일이 생성되었습니다: {output_path}")

JSON 파일이 생성되었습니다: data_generated.json


# 고2 데이터

In [13]:
folder_path_2 = "/Users/seunghyun/Library/CloudStorage/OneDrive-고려대학교/024.에세이 글 평가 데이터/주장/고2"

data2 = []

for filename in os.listdir(folder_path_2):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path_2, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
            data2.append(json_data)

print(f'{len(data2)}개의 파일이 성공적으로 읽혔습니다.')
data_list_2 = []

for filename in os.listdir(folder_path_2):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path_2, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)

            # 필요한 정보 추출
            paragraph_txt = json_data['paragraph'][0]['paragraph_txt']
            essay_prompt = json_data['info']['essay_prompt']
            essay_len = json_data['info']['essay_len']
            essay_main_subject = json_data['info']['essay_main_subject']

            # 데이터를 리스트에 추가
            data_list_2.append({
                'paragraph_txt': paragraph_txt,
                'essay_prompt': essay_prompt,
                'essay_len': essay_len,
                'essay_main_subject': essay_main_subject
            })

978개의 파일이 성공적으로 읽혔습니다.


In [14]:
# 리스트를 데이터프레임으로 변환
df_2 = pd.DataFrame(data_list_2)
df_2['essay_len'].describe()


count     978.000000
mean      935.470348
std       214.764201
min       461.000000
25%       866.000000
50%       894.000000
75%       949.750000
max      4833.000000
Name: essay_len, dtype: float64

- 고 2 분량 1200자 정도

In [15]:
df_2

,paragraph_txt,essay_prompt,essay_len,essay_main_subject
0,코로나 팬데믹이 시작된 이후 전 세계적으로 인종차별 사례가 늘어나고 있다.#@문장구...,최근 중국발 코로나바이러스로 인해 동양인에 대한 혐오감을 나타내는 서양인들이 늘어...,860,인종차별에 대한 본인의 생각
1,인종차별이란 단어를 들으면 생각나는 책이 있다. #@문장구분#바로 '검은 후드티 소...,최근 중국발 코로나바이러스로 인해 동양인에 대한 혐오감을 나타내는 서양인들이 늘어...,880,인종차별에 대한 본인의 생각
2,인종차별의 원인은 옛날의 노예제도로 거슬러 올라가야 합니다.#@문장구분#민주주의가 ...,최근 중국발 코로나바이러스로 인해 동양인에 대한 혐오감을 나타내는 서양인들이 늘어...,837,인종차별에 대한 본인의 생각
3,2020년 초 전 세계에서는 코로나바이러스가 퍼졌습니다.#@문장구분# 2020년에 ...,"지적재산권이란 특허권, 실용신안권, 상표권, 디자인권을 총칭하는 개념으로 지적재산...",811,지적 재산권에 대한 본인의 생각 작성
4,태어날 때부터 동양인에 대한 혐오감을 가지고 태어나서 그런 것이 아니라 동양인은 그...,최근 중국발 코로나바이러스로 인해 동양인에 대한 혐오감을 나타내는 서양인들이 늘어...,932,인종차별에 대한 본인의 생각
...,...,...,...,...
973,독일이 조 바이든 대통령이 지지한 '코로나19 백신 지적재산권 포기'에 대해 반대하...,"지적재산권이란 특허권, 실용신안권, 상표권, 디자인권을 총칭하는 개념으로 지적재산...",902,지적 재산권에 대한 본인의 생각 작성
974,인종차별에 대한 문제는 역사 속에 우리와 함께 존재한다.#@문장구분# 대표적인 인종...,최근 중국발 코로나바이러스로 인해 동양인에 대한 혐오감을 나타내는 서양인들이 늘어...,879,인종차별에 대한 본인의 생각
975,"얼마전만 해도 미국에서는 ""Black Lives Matter""라는 아프리카계 미국인...",최근 중국발 코로나바이러스로 인해 동양인에 대한 혐오감을 나타내는 서양인들이 늘어...,1039,인종차별에 대한 본인의 생각
976,코로나 19 팬데믹이 지구적 수준에서 유행한 지 2년이 넘어가고 있습니다.#@문장구...,"지적재산권이란 특허권, 실용신안권, 상표권, 디자인권을 총칭하는 개념으로 지적재산...",975,지적 재산권에 대한 본인의 생각 작성


## GPT4o-mini setting

In [ ]:
from openai import OpenAI
import os

## Set the API key and model name
MODEL="gpt-4o-mini"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "YOUR_KEY"))

In [111]:
completion = client.chat.completions.create(
  model=MODEL,
  messages=[
    {"role": "system", "content": "You are a helpful assistant. Help me with my math homework!"}, # <-- This is the system message that provides context to the model
    {"role": "user", "content": "Hello! Could you solve 2+2?"}  # <-- This is the user message for which the model will generate a response
  ]
)

print("Assistant: " + completion.choices[0].message.content)

Assistant: Of course! \( 2 + 2 = 4 \).


In [17]:
essay2 = df_2
high2 = essay2[:100]
len(high2)

100

In [18]:
high2.head()

,paragraph_txt,essay_prompt,essay_len,essay_main_subject
0,코로나 팬데믹이 시작된 이후 전 세계적으로 인종차별 사례가 늘어나고 있다.#@문장구...,최근 중국발 코로나바이러스로 인해 동양인에 대한 혐오감을 나타내는 서양인들이 늘어...,860,인종차별에 대한 본인의 생각
1,인종차별이란 단어를 들으면 생각나는 책이 있다. #@문장구분#바로 '검은 후드티 소...,최근 중국발 코로나바이러스로 인해 동양인에 대한 혐오감을 나타내는 서양인들이 늘어...,880,인종차별에 대한 본인의 생각
2,인종차별의 원인은 옛날의 노예제도로 거슬러 올라가야 합니다.#@문장구분#민주주의가 ...,최근 중국발 코로나바이러스로 인해 동양인에 대한 혐오감을 나타내는 서양인들이 늘어...,837,인종차별에 대한 본인의 생각
3,2020년 초 전 세계에서는 코로나바이러스가 퍼졌습니다.#@문장구분# 2020년에 ...,"지적재산권이란 특허권, 실용신안권, 상표권, 디자인권을 총칭하는 개념으로 지적재산...",811,지적 재산권에 대한 본인의 생각 작성
4,태어날 때부터 동양인에 대한 혐오감을 가지고 태어나서 그런 것이 아니라 동양인은 그...,최근 중국발 코로나바이러스로 인해 동양인에 대한 혐오감을 나타내는 서양인들이 늘어...,932,인종차별에 대한 본인의 생각


### 고2 paraphrase

In [114]:
paraphrase_prompt = '''
아래의 글을 정제된 언어로 paraphrase 해줘.
--------------------
{text}
'''

In [106]:
original_texts = []
paraphrased_texts = []

# 각 텍스트에 대해 paraphrase 실행
for text in high2['paragraph_txt']:
    # 텍스트를 paraphrase_prompt에 삽입
    user_prompt = paraphrase_prompt.format(text=text)

    # API 요청 생성
    completion = client.chat.completions.create(
      model=MODEL,
      messages=[
        {"role": "system", "content": paraphrase_prompt},
        {"role": "user", "content": user_prompt}
      ]
    )

    # 결과 저장
    original_texts.append(text)
    paraphrased_texts.append(completion.choices[0].message.content)

# DataFrame 생성
df = pd.DataFrame({
    'Original Text': original_texts,
    'Paraphrased Text': paraphrased_texts
})
df.to_csv("paraphrase_high2")

In [128]:
df['Original Text'] = df['Original Text'].replace(r'#@문장구분#', '', regex=True)


In [129]:
df

,Original Text,Paraphrased Text
0,코로나 팬데믹이 시작된 이후 전 세계적으로 인종차별 사례가 늘어나고 있다. 그리고 ...,"코로나19 팬데믹 이후, 전 세계적으로 인종차별 사례가 증가하고 있다. 그 중 하나..."
1,인종차별이란 단어를 들으면 생각나는 책이 있다. 바로 '검은 후드티 소년' 이라는 ...,"인종차별이라는 주제를 언급할 때, '검은 후드티 소년'이라는 책이 떠오른다. 이 책..."
2,인종차별의 원인은 옛날의 노예제도로 거슬러 올라가야 합니다.민주주의가 가장 발달했다...,인종차별의 기원은 과거의 노예제도로 거슬러 올라갑니다. 민주주의가 가장 발전했다고 ...
3,2020년 초 전 세계에서는 코로나바이러스가 퍼졌습니다. 2020년에 퍼진 코로나바...,"2020년 초, 전 세계적으로 코로나바이러스가 확산되었습니다. 이 바이러스는 202..."
4,태어날 때부터 동양인에 대한 혐오감을 가지고 태어나서 그런 것이 아니라 동양인은 그...,"동양인에 대한 혐오감은 태어날 때부터 존재한 것이 아니라, 그들이 경제적, 문화적 ..."
...,...,...
95,공공재란 뭘까? 공공재란 모든 사람이 공동으로 이용할 수 있는 재화나 서비스를 뜻한...,공공재란 무엇인가? 공공재는 모든 사람이 공동으로 이용할 수 있는 재화나 서비스를 ...
96,저는 지적 재산권이 꼭 보장되어야 한다고 생각합니다. 저는 최근 코로나19로 인해 ...,저는 지적 재산권이 반드시 보호되어야 한다고 생각합니다. 최근 코로나19와 관련하여...
97,코로나 백신 지적 재산권 절대 유예하지 않을 것 같습니다. 먼저 예를 들어보겠습니다...,코로나 백신의 지적 재산권은 절대적으로 유예되지 않을 것으로 예상됩니다. 예를 들어...
98,전세계적으로 코로나 감염확산 위험에 빠져 많은 사람들이 확진되고 죽으며 아주 심각한...,전세계적으로 코로나19의 감염 확산 우려로 인해 많은 사람들이 감염되거나 사망하며 ...


In [130]:
len(df)

100

In [131]:
df.to_csv("paraphrase_high2")

### 고2 correction

In [22]:
correction_prompt = '''
아래의 글에서 띄어쓰기 및 맞춤법 오류가 있는지 확인해보고, 있는 경우 수정해줘. 그리고 수정된 글을 반환해줘.
하나라도 수정한 글에는 글 앞에 '아래 글에서 맞춤법 오류를 수정한 내용입니다.\n\n-----'를 적어줘.
--------------------
{text}
'''

In [119]:
original_texts = []
correction_texts = []

# 각 텍스트에 대해 paraphrase 실행
for text in high2['paragraph_txt']:
    # 텍스트를 paraphrase_prompt에 삽입
    user_prompt = correction_prompt.format(text=text)

    # API 요청 생성
    completion = client.chat.completions.create(
      model=MODEL,
      messages=[
        {"role": "system", "content": correction_prompt},
        {"role": "user", "content": user_prompt}
      ],

    temperature = 0.7
    )

    # 결과 저장
    original_texts.append(text)
    correction_texts.append(completion.choices[0].message.content)

# DataFrame 생성
high2_df2 = pd.DataFrame({
    'Original Text': original_texts,
    'Correction Text': correction_texts
})

high2_df2['Correction Text'] = high2_df2['Correction Text'].str.replace(r'아래 글에서 맞춤법 오류를 수정한 내용입니다.\n\n-----', '', regex=True)

high2_df2.to_csv("correction_high2.csv", index = False)

In [23]:
high2_df2 = pd.read_csv("/Users/seunghyun/Downloads/correction_high2.csv")

In [24]:
high2_df2.iloc[73]

Original Text      남아프리카 공화국, 인도 등의 수십 개의 개발도상국 정부들은 코로나19 퇴치를 위해...
Correction Text    남아프리카 공화국, 인도 등의 수십 개의 개발도상국 정부들은 코로나19 퇴치를 위해...
Name: 73, dtype: object

In [25]:
# Correction Text 컬럼에서 \n 제거
high2_df2['Correction Text'] = high2_df2['Correction Text'].str.replace(r'\n', '', regex=True)

# 결과 저장
high2_df2.to_csv("correction_high2.csv", index=False)

In [26]:
import re

# @#문장구분# 이 포함된 부분 삭제
high2_df2['Original Text'] = high2_df2['Original Text'].replace(r'#@문장구분#', '', regex=True)

high2_df2.head()
high2_df2.to_csv("correction_high2.csv", index=False)


In [27]:
len(high2_df2)

100

In [28]:
# merge

paraphrase_high2 = pd.read_csv('/Users/seunghyun/Downloads/paraphrase_high2.csv')  
correction_high2 = pd.read_csv('/Users/seunghyun/Downloads/correction_high2.csv')  

merged_df = pd.merge(paraphrase_high2, correction_high2, on='Original Text', how='inner')

In [29]:
merged_df

,Unnamed: 0,Original Text,Paraphrased Text,Correction Text
0,0,코로나 팬데믹이 시작된 이후 전 세계적으로 인종차별 사례가 늘어나고 있다. 그리고 ...,"코로나19 팬데믹 이후, 전 세계적으로 인종차별 사례가 증가하고 있다. 그 중 하나...",코로나 팬데믹이 시작된 이후 전 세계적으로 인종차별 사례가 늘어나고 있다. 그리고 ...
1,1,인종차별이란 단어를 들으면 생각나는 책이 있다. 바로 '검은 후드티 소년' 이라는 ...,"인종차별이라는 주제를 언급할 때, '검은 후드티 소년'이라는 책이 떠오른다. 이 책...",인종차별이란 단어를 들으면 생각나는 책이 있다. 바로 '검은 후드티 소년'이라는 책...
2,2,인종차별의 원인은 옛날의 노예제도로 거슬러 올라가야 합니다.민주주의가 가장 발달했다...,인종차별의 기원은 과거의 노예제도로 거슬러 올라갑니다. 민주주의가 가장 발전했다고 ...,인종차별의 원인은 옛날의 노예제도로 거슬러 올라가야 합니다. #@문장구분# 민주주의...
3,3,2020년 초 전 세계에서는 코로나바이러스가 퍼졌습니다. 2020년에 퍼진 코로나바...,"2020년 초, 전 세계적으로 코로나바이러스가 확산되었습니다. 이 바이러스는 202...",2020년 초 전 세계에서는 코로나바이러스가 퍼졌습니다. 2020년에 퍼진 코로나바...
4,4,태어날 때부터 동양인에 대한 혐오감을 가지고 태어나서 그런 것이 아니라 동양인은 그...,"동양인에 대한 혐오감은 태어날 때부터 존재한 것이 아니라, 그들이 경제적, 문화적 ...",태어날 때부터 동양인에 대한 혐오감을 가지고 태어나서 그런 것이 아니라 동양인은 그...
...,...,...,...,...
95,95,공공재란 뭘까? 공공재란 모든 사람이 공동으로 이용할 수 있는 재화나 서비스를 뜻한...,공공재란 무엇인가? 공공재는 모든 사람이 공동으로 이용할 수 있는 재화나 서비스를 ...,공공재란 뭘까? 공공재란 모든 사람이 공동으로 이용할 수 있는 재화나 서비스를 뜻한...
96,96,저는 지적 재산권이 꼭 보장되어야 한다고 생각합니다. 저는 최근 코로나19로 인해 ...,저는 지적 재산권이 반드시 보호되어야 한다고 생각합니다. 최근 코로나19와 관련하여...,저는 지적 재산권이 꼭 보장되어야 한다고 생각합니다. 저는 최근 코로나19로 인해 ...
97,97,코로나 백신 지적 재산권 절대 유예하지 않을 것 같습니다. 먼저 예를 들어보겠습니다...,코로나 백신의 지적 재산권은 절대적으로 유예되지 않을 것으로 예상됩니다. 예를 들어...,코로나 백신 지적 재산권을 절대 유예하지 않을 것 같습니다. #@문장구분#먼저 예를...
98,98,전세계적으로 코로나 감염확산 위험에 빠져 많은 사람들이 확진되고 죽으며 아주 심각한...,전세계적으로 코로나19의 감염 확산 우려로 인해 많은 사람들이 감염되거나 사망하며 ...,전 세계적으로 코로나 감염 확산 위험에 빠져 많은 사람들이 확진되고 죽으며 아주 심...


In [30]:
merged_df = merged_df.drop('Unnamed: 0', axis=1)

In [31]:
merged_df

,Original Text,Paraphrased Text,Correction Text
0,코로나 팬데믹이 시작된 이후 전 세계적으로 인종차별 사례가 늘어나고 있다. 그리고 ...,"코로나19 팬데믹 이후, 전 세계적으로 인종차별 사례가 증가하고 있다. 그 중 하나...",코로나 팬데믹이 시작된 이후 전 세계적으로 인종차별 사례가 늘어나고 있다. 그리고 ...
1,인종차별이란 단어를 들으면 생각나는 책이 있다. 바로 '검은 후드티 소년' 이라는 ...,"인종차별이라는 주제를 언급할 때, '검은 후드티 소년'이라는 책이 떠오른다. 이 책...",인종차별이란 단어를 들으면 생각나는 책이 있다. 바로 '검은 후드티 소년'이라는 책...
2,인종차별의 원인은 옛날의 노예제도로 거슬러 올라가야 합니다.민주주의가 가장 발달했다...,인종차별의 기원은 과거의 노예제도로 거슬러 올라갑니다. 민주주의가 가장 발전했다고 ...,인종차별의 원인은 옛날의 노예제도로 거슬러 올라가야 합니다. #@문장구분# 민주주의...
3,2020년 초 전 세계에서는 코로나바이러스가 퍼졌습니다. 2020년에 퍼진 코로나바...,"2020년 초, 전 세계적으로 코로나바이러스가 확산되었습니다. 이 바이러스는 202...",2020년 초 전 세계에서는 코로나바이러스가 퍼졌습니다. 2020년에 퍼진 코로나바...
4,태어날 때부터 동양인에 대한 혐오감을 가지고 태어나서 그런 것이 아니라 동양인은 그...,"동양인에 대한 혐오감은 태어날 때부터 존재한 것이 아니라, 그들이 경제적, 문화적 ...",태어날 때부터 동양인에 대한 혐오감을 가지고 태어나서 그런 것이 아니라 동양인은 그...
...,...,...,...
95,공공재란 뭘까? 공공재란 모든 사람이 공동으로 이용할 수 있는 재화나 서비스를 뜻한...,공공재란 무엇인가? 공공재는 모든 사람이 공동으로 이용할 수 있는 재화나 서비스를 ...,공공재란 뭘까? 공공재란 모든 사람이 공동으로 이용할 수 있는 재화나 서비스를 뜻한...
96,저는 지적 재산권이 꼭 보장되어야 한다고 생각합니다. 저는 최근 코로나19로 인해 ...,저는 지적 재산권이 반드시 보호되어야 한다고 생각합니다. 최근 코로나19와 관련하여...,저는 지적 재산권이 꼭 보장되어야 한다고 생각합니다. 저는 최근 코로나19로 인해 ...
97,코로나 백신 지적 재산권 절대 유예하지 않을 것 같습니다. 먼저 예를 들어보겠습니다...,코로나 백신의 지적 재산권은 절대적으로 유예되지 않을 것으로 예상됩니다. 예를 들어...,코로나 백신 지적 재산권을 절대 유예하지 않을 것 같습니다. #@문장구분#먼저 예를...
98,전세계적으로 코로나 감염확산 위험에 빠져 많은 사람들이 확진되고 죽으며 아주 심각한...,전세계적으로 코로나19의 감염 확산 우려로 인해 많은 사람들이 감염되거나 사망하며 ...,전 세계적으로 코로나 감염 확산 위험에 빠져 많은 사람들이 확진되고 죽으며 아주 심...


In [32]:
merged_df.to_csv("merged_high2.csv", index=False)

### 고2 generation

In [19]:
essay_prompt_high2 = '''
최근 중국발 코로나바이러스로 인해 동양인에 대한 혐오감을 나타내는 서양인들이 늘어나고 있습니다. 그로 인해, 매스컴이나 SNS에서 인종차별을 받아 힘들어하는 동양인들을 자주 목격할 수 있습니다.
하지만, 최근에만 인종차별이 존재했을까요? 아닙니다. 아주 오래전부터 다른 인종에 대한 차별은 있었습니다.
우리나라에서는 인종차별을 안 하고 있을까요? 아닙니다. 우리나라에서도 알게 모르게 많은 부분에서 다른 국가의 사람들에 대해 인종차별을 하고 있으며 누군가에게 상처를 주고 있습니다.
즉, 인종차별은 누군가의 문제가 아니라 우리 모두의 문제입니다.
여러분들은 인종차별을 당해보신 적 있나요? 인종차별을 하는 사람들에 대해 어떻게 생각하시나요? 왜 생겨났다고 생각하나요? 인종차별을 근절하기 위해서는 어떻게 해야 할까요? 인종차별을 하는 사람들에 대해 전 세계적으로 처벌의 강도를 높여야 할까요?
인종차별을 하는 사람들에 대한 본인의 생각을 작성하고 인종차별은 해결할 수 있는 문제일지 아니면 해결하지 어려운 문제일지 작성해주세요. 또한, 그렇게 생각한 이유와 본인이 생각한 해결방법 혹은 해결할 수 없다면 왜 해결할 수 없다고 생각하는지 작성해주세요.'''

In [20]:
agent_prompt_high2 = '''
너는 고등학교 2학년 학생이고, 주장과 그에 대한 근거를 높은 수준에서 제시할 수 있어.
다음 주제로 1000자 글짓기를 하고 있어. 주제에 대해 너의 근거를 들어 주장하는 글쓰기를 해줘.
제목은 작성하지 말고, 본문만 작성해줘.
'''

In [21]:
len(high2)

100

In [33]:
#generation

generated_texts = []
for text in high2['paragraph_txt']:  
    completion = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": agent_prompt_high2},
            {"role": "user", "content": essay_prompt_high2}
        ],
        temperature=1.0
    )
    generated_texts.append(completion.choices[0].message.content)

# 병합된 데이터프레임에 Generated Texts 컬럼 추가
merged_df['Generated Texts'] = generated_texts

# 저장 또는 데이터 확인
merged_df.to_csv('merged_high2.csv', index=False) 

In [34]:
merged_df

,Original Text,Paraphrased Text,Correction Text,Generated Texts
0,코로나 팬데믹이 시작된 이후 전 세계적으로 인종차별 사례가 늘어나고 있다. 그리고 ...,"코로나19 팬데믹 이후, 전 세계적으로 인종차별 사례가 증가하고 있다. 그 중 하나...",코로나 팬데믹이 시작된 이후 전 세계적으로 인종차별 사례가 늘어나고 있다. 그리고 ...,"인종차별은 인류 역사와 깊은 연관을 지닌 복잡한 문제입니다. 특히, 현재 우리는 중..."
1,인종차별이란 단어를 들으면 생각나는 책이 있다. 바로 '검은 후드티 소년' 이라는 ...,"인종차별이라는 주제를 언급할 때, '검은 후드티 소년'이라는 책이 떠오른다. 이 책...",인종차별이란 단어를 들으면 생각나는 책이 있다. 바로 '검은 후드티 소년'이라는 책...,"인종차별은 단순히 특정 집단에 대한 편견이나 혐오에서 비롯되는 것이 아니라, 사회의..."
2,인종차별의 원인은 옛날의 노예제도로 거슬러 올라가야 합니다.민주주의가 가장 발달했다...,인종차별의 기원은 과거의 노예제도로 거슬러 올라갑니다. 민주주의가 가장 발전했다고 ...,인종차별의 원인은 옛날의 노예제도로 거슬러 올라가야 합니다. #@문장구분# 민주주의...,"인종차별은 인류 역사 속에서 지속적으로 나타난 심각한 문제이며, 최근의 코로나19 ..."
3,2020년 초 전 세계에서는 코로나바이러스가 퍼졌습니다. 2020년에 퍼진 코로나바...,"2020년 초, 전 세계적으로 코로나바이러스가 확산되었습니다. 이 바이러스는 202...",2020년 초 전 세계에서는 코로나바이러스가 퍼졌습니다. 2020년에 퍼진 코로나바...,인종차별은 단지 개인의 잘못이 아니라 사회 전체가 공유하는 문제입니다. 역사적으로 ...
4,태어날 때부터 동양인에 대한 혐오감을 가지고 태어나서 그런 것이 아니라 동양인은 그...,"동양인에 대한 혐오감은 태어날 때부터 존재한 것이 아니라, 그들이 경제적, 문화적 ...",태어날 때부터 동양인에 대한 혐오감을 가지고 태어나서 그런 것이 아니라 동양인은 그...,인종차별은 인류가 오랜 역사를 거치며 직면해 온 가장 심각한 사회적 문제 중 하나입...
...,...,...,...,...
95,공공재란 뭘까? 공공재란 모든 사람이 공동으로 이용할 수 있는 재화나 서비스를 뜻한...,공공재란 무엇인가? 공공재는 모든 사람이 공동으로 이용할 수 있는 재화나 서비스를 ...,공공재란 뭘까? 공공재란 모든 사람이 공동으로 이용할 수 있는 재화나 서비스를 뜻한...,인종차별은 인류 역사 속에서 반복적으로 발생해온 심각한 사회 문제입니다. 최근 중국...
96,저는 지적 재산권이 꼭 보장되어야 한다고 생각합니다. 저는 최근 코로나19로 인해 ...,저는 지적 재산권이 반드시 보호되어야 한다고 생각합니다. 최근 코로나19와 관련하여...,저는 지적 재산권이 꼭 보장되어야 한다고 생각합니다. 저는 최근 코로나19로 인해 ...,"인종차별은 인류 역사 속에서 끊임없이 존재해온 문제이며, 그 근본 원인들은 복잡하고..."
97,코로나 백신 지적 재산권 절대 유예하지 않을 것 같습니다. 먼저 예를 들어보겠습니다...,코로나 백신의 지적 재산권은 절대적으로 유예되지 않을 것으로 예상됩니다. 예를 들어...,코로나 백신 지적 재산권을 절대 유예하지 않을 것 같습니다. #@문장구분#먼저 예를...,인종차별은 인류 역사 속에서 오랜 시간 동안 지속되어온 심각한 문제입니다. 과거부터...
98,전세계적으로 코로나 감염확산 위험에 빠져 많은 사람들이 확진되고 죽으며 아주 심각한...,전세계적으로 코로나19의 감염 확산 우려로 인해 많은 사람들이 감염되거나 사망하며 ...,전 세계적으로 코로나 감염 확산 위험에 빠져 많은 사람들이 확진되고 죽으며 아주 심...,"인종차별은 인류 역사상 끊임없이 존재해 온 문제이며, 지금도 여전히 많은 이들에게 ..."


In [ ]:
len(merged_df)

In [35]:
# to_json!

# paraphrased 전용

csv_path = '/Users/seunghyun/Downloads/merged_high2.csv'
merged_df = pd.read_csv(csv_path)

# JSON 파일 형식 생성
json_output = []

# CSV의 각 행을 반복하며 JSON 포맷으로 변환
json_output = {
    "original": merged_df['Correction Text'].tolist(),
    "sampled": merged_df['Paraphrased Text'].tolist()
}


# JSON 파일 저장
output_path = 'data_paraphrased_2.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(json_output, f, ensure_ascii=False, indent=4)

print(f"JSON 파일이 생성되었습니다: {output_path}")

JSON 파일이 생성되었습니다: data_paraphrased_2.json


In [36]:
# generation 전용

csv_path = '/Users/seunghyun/Downloads/merged_high2.csv'
merged_df = pd.read_csv(csv_path)

# JSON 파일 형식 생성
json_output = []

# CSV의 각 행을 반복하며 JSON 포맷으로 변환
json_output = {
    "original": merged_df['Correction Text'].tolist(),
    "sampled": merged_df['Generated Texts'].tolist()
}


# JSON 파일 저장
output_path = 'data_generated_2.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(json_output, f, ensure_ascii=False, indent=4)

print(f"JSON 파일이 생성되었습니다: {output_path}")

JSON 파일이 생성되었습니다: data_generated_2.json
